In [123]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20210104'
endDate = '20210107'

readPath = r'L:\orderLog\data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)


for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
try:
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r1, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])

try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders out of 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog['accCode'] // 100)
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog['sdd'] = orderLog.groupby('order')['sdd'].transform('first')
orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')
df = orderLog.copy()

['L:\\orderLog\\data\\20210104.pkl' 'L:\\orderLog\\data\\20210105.pkl'
 'L:\\orderLog\\data\\20210106.pkl' 'L:\\orderLog\\data\\20210107.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
346895,20210104,zs_66_01,6634,2000099,-1,1,48683.0,0,0,-1.0,1.812810e+10
391993,20210104,zs_66_01,6634,2000790,-1,1,53359.0,0,0,-1.0,1.812812e+10
456876,20210104,zs_66_01,6634,2002198,-1,1,40848.0,0,0,-1.0,1.812808e+10
472646,20210104,zs_66_01,6634,2002285,-1,7,52982.0,0,0,-1.0,-1.000000e+00
506588,20210104,zs_64_01,6480,2002439,-1,1,37021.0,0,0,-1.0,5.973100e+04
542462,20210104,zs_66_01,6634,2002616,-1,1,53765.0,0,0,-1.0,1.812812e+10
563491,20210104,zs_64_01,6480,2002730,-1,1,39833.0,0,0,-1.0,8.439700e+04
570242,20210104,zs_66_01,6634,2002771,-1,1,41296.0,0,0,-1.0,1.812808e+10
572440,20210104,zs_66_01,6634,2002780,-1,1,37490.0,0,0,-1.0,1.812805e+10
597756,20210104,zs_52_06,5273,2002893,-1,1,34913.0,0,0,-1.0,2.755240e+05


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
     secid  accCode      colo      vai  updateType         sdd  internalId  \
0  2300271     8856  zs_88_04  22400.0           0  93007340.0        30.0   
1  2300271     8856  zs_88_04  22400.0           2        -1.0        30.0   
2  2300271     8856  zs_88_04  22400.0           1     34209.0        30.0   
3  2300271     8856  zs_88_04  22400.0           3        -1.0        30.0   
4  2300271     8856  zs_88_04  22400.0           0  93009860.0        34.0   
5  2300271     8856  zs_88_04  22400.0           2        -1.0        34.0   
6  2300271     8856  zs_88_04  22400.0           2        -1.0        34.0   

   orderId  absOrderSize  absFilledThisUpdate  absOrderSizeCumFilled  \
0     30.0           100                    0                      0   
1     30.0           100                    0                      0   
2     30.0           100                    0                

In [53]:
orderLog['directNum'] = np.nan
orderLog['num1'] = np.where(orderLog['updateType'] == 4, 1, 0)
orderLog['num1'] = orderLog.groupby('order')['num1'].transform('sum')
checkLog = orderLog[(orderLog['num1'] >= 1) & (orderLog['updateType'] == 0)]
checkLog['directNum'] = checkLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
checkLog = orderLog[orderLog['order'].isin(checkLog['order'].unique())]
checkLog = checkLog[checkLog['updateType'] == 4]
checkLog['directNum'] = checkLog.groupby(['date', 'secid', 'vai', 'absFilledThisUpdate', 'tradePrice'])['orderDirection1'].transform('nunique')
checkLog[checkLog['directNum'] >= 2].groupby(['date', 'secid', 'vai', 'absFilledThisUpdate', 'tradePrice'])['accCode'].unique()

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


date      secid    vai         absFilledThisUpdate  tradePrice
20210104  1603596  710560.0    100                  36.35             [528701, 8943]
                   903060.0    100                  36.16             [527101, 8943]
          2002352  1407603.0   400                  89.43               [5281, 9461]
20210106  2002617  49893052.0  400                  9.09          [6623, 6627, 9208]
Name: accCode, dtype: object

In [55]:
orderLog[(orderLog['date'] == 20210104) & (orderLog['secid'] == 1603596) & (orderLog['vai'] == 903060)]\
[['date', 'secid', 'updateType', 'ApplSeqNum', 'sdd', 'vai', 'ars', 'absOrderSize', 'clockAtArrival', 'orderDirection', 'caamd', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mra100', 'mrb100']]

,date,secid,updateType,ApplSeqNum,sdd,vai,ars,absOrderSize,clockAtArrival,orderDirection,caamd,absFilledThisUpdate,orderPrice,tradePrice,mra100,mrb100
627570,20210104,1603596,0,0.0,94935000.0,903060.0,1.0,1900,1609724977782008,-1,1609724977781859,0,36.16,-1.00,3617.0,3616.0
627571,20210104,1603596,2,0.0,94935000.0,903060.0,1.0,1900,1609724977987224,-1,1609724977781859,0,36.16,-1.00,3617.0,3616.0
627572,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724978227410,-1,1609724977781859,200,36.16,36.16,3616.0,3612.0
627573,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724978227511,-1,1609724977781859,1700,36.16,36.16,3616.0,3612.0
647199,20210104,1603596,0,0.0,94935000.0,903060.0,1.0,1900,1609724977142120,-1,1609724977142057,0,36.16,-1.00,3617.0,3616.0
647200,20210104,1603596,2,0.0,94935000.0,903060.0,1.0,1900,1609724977292900,-1,1609724977142057,0,36.16,-1.00,3617.0,3616.0
647201,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724977549602,-1,1609724977142057,100,36.16,36.16,3616.0,3612.0
647202,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724977751315,-1,1609724977142057,200,36.16,36.16,3616.0,3612.0
647203,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724977953935,-1,1609724977142057,100,36.16,36.16,3616.0,3612.0
647204,20210104,1603596,1,0.0,94935000.0,903060.0,1.0,1900,1609724980475370,-1,1609724977142057,0,36.16,-1.00,3615.0,3612.0


In [58]:
orderLog['type'] = np.where((orderLog['orderDirection'].isin([1, 2]) & ((orderLog['orderPrice'] * 100).astype(int) >= orderLog['mra100'])) | \
                           (orderLog['orderDirection'].isin([-1, -2]) & ((orderLog['orderPrice'] * 100).astype(int) <= orderLog['mrb100'])), 'aggressive', 'passive')
orderLog['type'] = orderLog.groupby('order')['type'].transform('first')
orderLog[(orderLog['date'] == 20210104) & (orderLog['secid'] == 1603596) & (orderLog['vai'] == 903060)]\
[['date', 'secid', 'updateType', 'ApplSeqNum', 'sdd', 'vai', 'ars', 'absOrderSize', 'clockAtArrival', 'orderDirection', 'caamd', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mra100', 'mrb100', 'order', 'type']]

,date,secid,updateType,ApplSeqNum,sdd,vai,ars,absOrderSize,clockAtArrival,orderDirection,caamd,absFilledThisUpdate,orderPrice,tradePrice,mra100,mrb100,order,type
627570,20210104,1603596,0,0.0,94935000.0,903060.0,1.0,1900,1609724977782008,-1,1609724977781859,0,36.16,-1.00,3617.0,3616.0,154738,aggressive
627571,20210104,1603596,2,0.0,94935000.0,903060.0,1.0,1900,1609724977987224,-1,1609724977781859,0,36.16,-1.00,3617.0,3616.0,154738,aggressive
627572,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724978227410,-1,1609724977781859,200,36.16,36.16,3616.0,3612.0,154738,aggressive
627573,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724978227511,-1,1609724977781859,1700,36.16,36.16,3616.0,3612.0,154738,aggressive
647199,20210104,1603596,0,0.0,94935000.0,903060.0,1.0,1900,1609724977142120,-1,1609724977142057,0,36.16,-1.00,3617.0,3616.0,159165,aggressive
647200,20210104,1603596,2,0.0,94935000.0,903060.0,1.0,1900,1609724977292900,-1,1609724977142057,0,36.16,-1.00,3617.0,3616.0,159165,aggressive
647201,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724977549602,-1,1609724977142057,100,36.16,36.16,3616.0,3612.0,159165,aggressive
647202,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724977751315,-1,1609724977142057,200,36.16,36.16,3616.0,3612.0,159165,aggressive
647203,20210104,1603596,4,0.0,94935000.0,903060.0,1.0,1900,1609724977953935,-1,1609724977142057,100,36.16,36.16,3616.0,3612.0,159165,aggressive
647204,20210104,1603596,1,0.0,94935000.0,903060.0,1.0,1900,1609724980475370,-1,1609724977142057,0,36.16,-1.00,3615.0,3612.0,159165,aggressive


In [125]:
orderLog['type'] = np.where((orderLog['orderDirection'].isin([1, 2]) & ((orderLog['orderPrice'] * 100).astype(int) >= orderLog['mra100'])) | \
                           (orderLog['orderDirection'].isin([-1, -2]) & ((orderLog['orderPrice'] * 100).astype(int) <= orderLog['mrb100'])), 'aggressive', 'passive')
orderLog['type'] = orderLog.groupby('order')['type'].transform('first')
checkLog = orderLog[(orderLog['updateType'] == 4)]
checkLog['num1'] = checkLog.groupby(['date', 'secid', 'tradePrice', 'absFilledThisUpdate'])['orderDirection1'].transform('nunique')
checkLog['num2'] = checkLog.groupby(['date', 'secid', 'tradePrice', 'absFilledThisUpdate'])['type'].transform('nunique')
checkLog = checkLog[(checkLog['num1'] == 2) & (checkLog['num2'] == 2)]
# checkLog['t1'] = np.where(checkLog['type'] == 'aggressive', checkLog['clockAtArrival'], np.nan)
# checkLog['t2'] = np.where(checkLog['type'] == 'passive', checkLog['clockAtArrival'], np.nan)
# checkLog['t1'] = checkLog.groupby(['date', 'secid', 'tradePrice', 'absFilledThisUpdate'])['t1'].transform('max')
# checkLog['t2'] = checkLog.groupby(['date', 'secid', 'tradePrice', 'absFilledThisUpdate'])['t2'].transform('min')
# checkLog = checkLog[checkLog['t1'] > checkLog['t2']]
checkLog['o1'] = checkLog.groupby(['date', 'secid', 'tradePrice', 'absFilledThisUpdate']).grouper.group_info[0]
l1 = []
l2 = []
for group, groupData in checkLog.groupby(['o1']):
    groupData['index'] = groupData.index
    groupData1 = groupData[groupData['type'] == 'aggressive'][['index', 'type', 'clockAtArrival', 'duration', 'startClock', 'order', 'orderDirection1']]
    groupData2 = groupData[groupData['type'] == 'passive'][['index', 'type', 'clockAtArrival', 'duration', 'startClock', 'order', 'orderDirection1']]

    for i in range(groupData1.shape[0]):
        for j in range(groupData2.shape[0]):
            data1 = groupData1.iloc[i, :]
            data2 = groupData2.iloc[j, :]
            if (data2['duration'] > data1['duration']) & (data1['startClock'] > data2['startClock']) & \
            (abs(data2['clockAtArrival'] - data1['clockAtArrival']) <= 100000) & (data1['orderDirection1'] != data2['orderDirection1']):
                l1.append(data1['index']) 
                l2.append(data2['index'])
l1

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[2166000]

In [126]:
checkLog.loc[l1 + l2, :][['o1', 'date', 'secid', 'updateType', 'ApplSeqNum', 'sdd', 'vai', 'ars', 'absOrderSize', 'clockAtArrival', 'orderDirection', 'caamd', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mra100', 'mrb100', 'order', 'type', 'duration', 'startClock', 'clockAtArrival']]

,o1,date,secid,updateType,ApplSeqNum,sdd,vai,ars,absOrderSize,clockAtArrival,orderDirection,caamd,absFilledThisUpdate,orderPrice,tradePrice,mra100,mrb100,order,type,duration,startClock,clockAtArrival
2166000,615,20210106,1603712,4,0.0,100010000.0,1914151.0,1.0,500,1609898411276635,-1,1609898410840142,100,43.15,43.15,4318.0,4315.0,534298,aggressive,436421,1609898410840214,1609898411276635
1949951,615,20210106,1603712,4,0.0,95950000.0,1907851.0,2.0,100,1609898411328203,1,1609898390261052,100,43.15,43.15,4318.0,4315.0,484367,passive,21066565,1609898390261638,1609898411328203


In [127]:
orderLog[orderLog['order'].isin([534298, 484367])][['date', 'accCode', 'secid', 'updateType', 'vai', 'ars', 'absOrderSize', 'clockAtArrival', 'orderDirection', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mra100', 'mrb100', 'order']]

,date,accCode,secid,updateType,vai,ars,absOrderSize,clockAtArrival,orderDirection,absFilledThisUpdate,orderPrice,tradePrice,mra100,mrb100,order
1949948,20210106,6282,1603712,0,1907851.0,2.0,100,1609898390261638,1,0,43.15,-1.00,4330.0,4314.0,484367
1949949,20210106,6282,1603712,2,1907851.0,2.0,100,1609898390298610,1,0,43.15,-1.00,4330.0,4314.0,484367
1949950,20210106,6282,1603712,1,1907851.0,2.0,100,1609898410834348,1,0,43.15,-1.00,4318.0,4315.0,484367
1949951,20210106,6282,1603712,4,1907851.0,2.0,100,1609898411328203,1,100,43.15,43.15,4318.0,4315.0,484367
2165998,20210106,537403,1603712,0,1914151.0,1.0,500,1609898410840214,-1,0,43.15,-1.00,4318.0,4315.0,534298
2165999,20210106,537403,1603712,2,1914151.0,1.0,500,1609898411024783,-1,0,43.15,-1.00,4318.0,4315.0,534298
2166000,20210106,537403,1603712,4,1914151.0,1.0,500,1609898411276635,-1,100,43.15,43.15,4318.0,4315.0,534298
2166001,20210106,537403,1603712,4,1914151.0,1.0,500,1609898411276694,-1,400,43.15,43.15,4318.0,4315.0,534298


In [ ]:
checkLog.loc[l1 + l2, :][['o1', 'date', 'secid', 'updateType', 'ApplSeqNum', 'sdd', 'vai', 'ars', 'absOrderSize', 'clockAtArrival', 'orderDirection', 'caamd', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mra100', 'mrb100', 'order', 'type', 'duration', 'startClock', 'clockAtArrival']]

In [73]:
1609741405495123 - 1609736759042678	

4646452445

In [106]:
orderLog[orderLog['order'].isin([420964, 482867])][['date', 'secid', 'updateType', 'ApplSeqNum', 'sdd', 'vai', 'ars', 'absOrderSize', 'clockAtArrival', 'orderDirection', 'caamd', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mra100', 'mrb100', 'order']]

,date,secid,updateType,ApplSeqNum,sdd,vai,ars,absOrderSize,clockAtArrival,orderDirection,caamd,absFilledThisUpdate,orderPrice,tradePrice,mra100,mrb100,order
1711671,20210106,2000547,0,22533584.0,140100580.0,35887066.0,1.0,900,1609912860578057,1,1609912860577991,0,30.51,-1.00,3051.0,3050.0,420964
1711672,20210106,2000547,2,0.0,140100580.0,35887066.0,1.0,900,1609912860583173,1,1609912860577991,0,30.51,-1.00,3060.0,3051.0,420964
1711673,20210106,2000547,4,0.0,140100580.0,35887066.0,1.0,900,1609912860619533,1,1609912860577991,800,30.51,30.51,3060.0,3051.0,420964
1711674,20210106,2000547,4,0.0,140100580.0,35887066.0,1.0,900,1609912860623739,1,1609912860577991,100,30.51,30.51,3051.0,3050.0,420964
1943352,20210106,2000547,0,22525649.0,140054100.0,35866566.0,2.0,100,1609912854128691,1,1609912854128610,0,30.51,-1.00,3058.0,3050.0,482867
1943353,20210106,2000547,2,0.0,140054100.0,35866566.0,2.0,100,1609912854156032,1,1609912854128610,0,30.51,-1.00,3058.0,3050.0,482867
1943354,20210106,2000547,4,0.0,140054100.0,35866566.0,2.0,100,1609912860627201,1,1609912854128610,100,30.51,30.51,3060.0,3051.0,482867
